# This scripts does the following to rectify the player and ball tracking data
(1) player tracking df: have all plays go R->L; remove rows with no ball info  
(2) master df: Add column for snap ball pos; remove rows with no ball info; add RTLyardline  
(3) ball tracking: add/remove rows before/after to align with player tracking; flip coords if necessary  

In [21]:
import pandas as pd, numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [22]:
pds = pd.read_pickle("../data/master_player_df.pkl")
md = pd.read_csv("../data/master_play_table_v1.csv", low_memory=False)
bd = pd.read_pickle("../data/balltrack.pkl")

For each play, find play direction by referencing QB and C positions...

In [ ]:
import pandasql as ps;
q = ("select gameId, playId from pds " +
     "group by gameId, playId " +
     "having sum(case when position = 'QB' then 1 else 0 end) = 1 and " +
     "sum(case when position = 'C' then 1 else 0 end) = 1")

In [24]:
pds_qb = pds.query("position == 'QB'")[['gameId', 'playId', 'coordinates']]
pds_qb['coordinates'] = [c[0][0] for c in pds_qb['coordinates']]

In [ ]:
#t = pds_qb.groupby(['gameId', 'playId']).size()
#t = t.loc[t==1]

In [25]:
pds_c = pds.query("position == 'C'")[['gameId', 'playId', 'coordinates']]
pds_c['ncoords']=[k.shape[0] for k in pds_c['coordinates']]
pds_c['coordinates']=[c[0][0] for c in pds_c['coordinates']]

In [26]:
qbcounts = pds_qb.groupby(['gameId', 'playId'])['coordinates'].count().reset_index()
qbcounts.rename(columns = {'coordinates': 'count'}, inplace = True)

In [27]:
c_counts = pds_c.groupby(['gameId', 'playId'])['coordinates'].count().reset_index()
c_counts.rename(columns = {'coordinates': 'count'}, inplace = True)

In [28]:
pds_qb1 = pd.merge(pds_qb, qbcounts, on = ['gameId', 'playId'], how = 'left')
pds_c1 = pd.merge(pds_c, c_counts, on = ['gameId', 'playId'], how = 'left')

In [31]:
pds_qb1 = pds_qb1[pds_qb1['count'] == 1]
pds_c1 = pds_c1[pds_c1['count'] == 1]

In [35]:
pds_m = pd.merge(pds_qb1, pds_c1, on=['gameId', 'playId'])

In [37]:
playdir = pds_m.eval("dir=1*(coordinates_x>coordinates_y)")[['gameId','playId','dir','ncoords']]

dir == 0: Going the 'wrong' way

Rectify ball tracking dataset

In [38]:
bd = pd.merge(bd, playdir)

In [39]:
#Rectify direction
bd['ballTrack']= [b if d == 1 else np.round(np.array([120,53.3])-b,2) for (b,d) in zip(bd.ballTrack, bd.dir)]

In [40]:
#rectify length of series
bd['ballTrack']=[np.vstack((np.ones([20,1])*(x[0,:].T), x))[:y] for (x,y) in zip(bd['ballTrack'], bd['ncoords'])]

In [41]:
bd.drop("ncoords", axis=1, inplace=True)

In [42]:
bd['snapPos']=[x[0,:] for x in bd['ballTrack']]
# bd.drop('ballTrack', axis=1, inplace=True)

Rectify player master

In [43]:
pds=pd.merge(left=pds, right=bd[['gameId', 'playId', 'dir', 'snapPos']])

In [44]:
#Rectify direction
pds['coordinates']= [b if d == 1 else np.round(np.array([120,53.3])-b,2) for (b,d) in zip(pds.coordinates, pds.dir)]

Rectify play master

In [45]:
md=pd.merge(left=md, right=bd[['gameId', 'playId', 'dir', 'snapPos']])

In [46]:
#Rectify direction
md['rectYardline']= [b if d == 1 else np.round(120-b,2) for (b,d) in zip(md.absoluteYardlineNumber, md.dir)]

In [47]:
pds.to_pickle("../data/player_play_rect.pkl")
md.to_pickle("../data/play_rect.pkl")
bd.to_pickle("../data/ball_rect.pkl")